In [ ]:

! pip install -U transformers 
! pip install -U datasets 
! pip install -U accelerate 
! pip install -U peft 
! pip install -U trl 
! pip install -U bitsandbytes 
 #! pip install -U wandb

In [1]:
import wandb

In [ ]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import (
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model,
)
import os, torch, wandb
from datasets import load_dataset
from trl import SFTTrainer, setup_chat_format

In [ ]:
from huggingface_hub import login
 
login(
  token="hf_Vb", # ADD YOUR TOKEN HERE
  add_to_git_credential=True
)
 

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (manager).
Your token has been saved to C:\Users\kosmo\.cache\huggingface\token
Login successful


In [4]:
base_model = "EleutherAI/pythia-2.8b"
dataset_name = "lavita/ChatDoctor-HealthCareMagic-100k"
new_model = "EleutherAI/pythia-2.8b-Healthchat-Doctor"


In [5]:
from huggingface_hub import model_info

info = model_info("EleutherAI/pythia-2.8b")
print(info)

ModelInfo(id='EleutherAI/pythia-2.8b', author='EleutherAI', sha='2a259cdd96a4beb1cdf467512e3904197345f6a9', created_at=datetime.datetime(2023, 2, 13, 14, 37, 12, tzinfo=datetime.timezone.utc), last_modified=datetime.datetime(2023, 6, 9, 0, 35, 37, tzinfo=datetime.timezone.utc), private=False, gated=False, disabled=False, downloads=24817, likes=28, library_name='transformers', tags=['transformers', 'pytorch', 'safetensors', 'gpt_neox', 'text-generation', 'causal-lm', 'pythia', 'en', 'dataset:EleutherAI/pile', 'arxiv:2304.01373', 'arxiv:2101.00027', 'arxiv:2201.07311', 'license:apache-2.0', 'autotrain_compatible', 'text-generation-inference', 'endpoints_compatible', 'region:us'], pipeline_tag='text-generation', mask_token=None, card_data={'base_model': None, 'datasets': ['EleutherAI/pile'], 'eval_results': None, 'language': ['en'], 'library_name': None, 'license': 'apache-2.0', 'license_name': None, 'license_link': None, 'metrics': None, 'model_name': None, 'pipeline_tag': None, 'tags': 

In [6]:
import torch
import flash_attn

In [7]:
 print(flash_attn.__version__)

2.7.0.post2


In [8]:
# Set torch dtype and attention implementation
if torch.cuda.get_device_capability()[0] >= 8:
     
    torch_dtype = torch.bfloat16
    attn_implementation = "flash_attention_2"
else:
    torch_dtype = torch.float16
    attn_implementation = "eager"

In [12]:
import torch
import gc
import os

def check_gpu_memory():
    # Nettoyage préventif
    torch.cuda.empty_cache()
    gc.collect()
    
    # Vérification détaillée
    if torch.cuda.is_available():
        print("\n=== État de la mémoire GPU ===")
        print(f"GPU utilisée : {torch.cuda.get_device_name(0)}")
        total = torch.cuda.get_device_properties(0).total_memory / 1024**3
        available = torch.cuda.mem_get_info()[0] / 1024**3
        used = total - available
        print(f"Mémoire totale   : {total:.2f} GB")
        print(f"Mémoire utilisée : {used:.2f} GB")
        print(f"Mémoire libre    : {available:.2f} GB")
        print("===========================\n")
        
        if available < 2:  # Alerte si moins de 2GB disponibles
            print("⚠️ ATTENTION : Mémoire GPU faible !")
    else:
        print("❌ Pas de GPU CUDA disponible")

# Utilisation
check_gpu_memory()


=== État de la mémoire GPU ===
GPU utilisée : NVIDIA GeForce RTX 4060
Mémoire totale   : 8.00 GB
Mémoire utilisée : 3.03 GB
Mémoire libre    : 4.96 GB



In [10]:
import torch

# Vérifier si CUDA est disponible
print("CUDA disponible:", torch.cuda.is_available())

# Vérifier quelle GPU est utilisée
if torch.cuda.is_available():
    print("GPU actuelle:", torch.cuda.get_device_name(0))
    print("Mémoire totale GPU:", torch.cuda.get_device_properties(0).total_memory / 1024**3, "GB")
    print("Mémoire disponible:", torch.cuda.mem_get_info()[0] / 1024**3, "GB")

# Libérer la mémoire GPU si nécessaire
torch.cuda.empty_cache()

CUDA disponible: True
GPU actuelle: NVIDIA GeForce RTX 4060
Mémoire totale GPU: 7.99560546875 GB
Mémoire disponible: 6.9326171875 GB


In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "EleutherAI/pythia-2.8b",
    torch_dtype=torch.float16,
    device_map="auto",
    low_cpu_mem_usage=True
)

In [11]:
# QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=True,
)

# Load model
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
    attn_implementation=attn_implementation
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)

In [ ]:
import bitsandbytes as bnb

def find_all_linear_names(model):
    cls = bnb.nn.Linear4bit
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names:  # needed for 16 bit
        lora_module_names.remove('lm_head')
    return list(lora_module_names)

In [ ]:
modules = find_all_linear_names(model)

In [ ]:
modules

In [ ]:
tokenizer.chat_template = None

In [ ]:
# LoRA config
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=modules
)
#model, tokenizer = setup_chat_format(model, tokenizer)
model = get_peft_model(model, peft_config)

In [ ]:
tokenizer

In [ ]:
from transformers import AutoTokenizer

# Initialiser le tokenizer avec votre modèle de base
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)

In [ ]:
#Importing the dataset
dataset = load_dataset(dataset_name, split="all")
dataset = dataset.shuffle(seed=65).select(range(1000))


In [ ]:
def format_chat_template(row, tokenizer=tokenizer):
    row_json = [
        {"role": "user", "content": row["instruction"] + "\n" + row["input"]},  # Combinons instruction et input
        {"role": "assistant", "content": row["output"]}
    ]
    row["text"] = tokenizer.apply_chat_template(row_json, tokenize=False)
    return row

dataset = dataset.map(
    format_chat_template,
    fn_kwargs={'tokenizer': tokenizer},
    num_proc=4,
)

In [ ]:
dataset

In [ ]:
dataset['text'][3]


In [ ]:
dataset = dataset.train_test_split(test_size=0.1)

In [ ]:
#Hyperparamter
training_arguments = TrainingArguments(
    output_dir=new_model,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    num_train_epochs=1,
    eval_strategy="steps",
    eval_steps=0.2,
    logging_steps=1,
    warmup_steps=10,
    logging_strategy="steps",
    learning_rate=2e-4,
    fp16=False,
    bf16=False,
    group_by_length=True,
    report_to="wandb"
)

In [ ]:
# Setting sft parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    peft_config=peft_config,
    max_seq_length= 512,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)

In [ ]:
#model.config.use_cache = False
trainer.train()